<a href="https://colab.research.google.com/github/fernandodeeke/hydroscheduling2025/blob/main/wind_fit_2025_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Wind Speed PDF Estimation: Comparison with Clamped Cubic Spline

import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
from numpy.polynomial import Chebyshev as Cheb
from scipy.integrate import quad
from scipy.stats import gaussian_kde
from scipy.interpolate import CubicSpline
from sympy import symbols, expand, integrate, solve, Eq, lambdify, latex, Piecewise, simplify
from ipywidgets import interact, IntSlider
import time
import os

# Load wind speed data
# filename = "offshore_pr150m_1994.txt"
# filename = "itaguacu110m_1994.txt"
#filename = "itaguacu150m_1994.txt"
# filename = "macambira70m_1995.txt"
#filename = "afranio80m_1994.txt"
#filename = "anta50m_2000.txt"
# wind_speed_data = np.loadtxt(filename)
filename = "MERRA_basic_W34.666_S07.000_1992.txt"
# Find the header row index by scanning for the line that starts with 'time'
with open(filename, 'r') as f:
    for header_row, line in enumerate(f):
        if line.lstrip().startswith("time"):
            break

# Load only the 5th column (wSpeed), skipping the preamble and header
wind_speed_data = np.loadtxt(
    filename,
    delimiter="\t",
    skiprows=header_row+1,
    usecols=(4,)
)

#filename = "vortex_time_series_56425-20y 100m UTC -3 CFSR.txt"

#  Find the first data line (starts with YYYYMMDD HHMM)
#with open(filename, 'r') as f:
#    for header_row, line in enumerate(f):
        # look for 8 digits, whitespace, 4 digits at the start
#        if re.match(r'^\s*\d{8}\s+\d{4}', line):
#            break

#  Load ONLY the 3rd column (zero-based index 2), skipping that many rows
#wind_speed_data = np.loadtxt(
#    filename,
#    skiprows=header_row,
#    usecols=(2,),        # the “M(m/s)” column
#    dtype=float
#)

# Chebyshev Moment Fitting with Normalization and Positivity
def FChebyshev(V, N):
    n = len(V)
    x, t = symbols('x t')
    f = [1, x]
    for i in range(2, N):
        f.append(expand(2 * x * f[i - 1] - f[i - 2]))
    Vmax = np.max(V)
    Vnorm = V / Vmax
    m = [(1 / n) * sum(Vnorm**(k - 1)) for k in range(1, N + 1)]
    beta = symbols(f'beta0:{N}')
    eqs = [Eq(sum(beta[k - 1] * integrate(x**(j - 1) * f[k - 1], (x, 0, 1))
                  for k in range(1, N + 1)), m[j - 1])
           for j in range(1, N + 1)]
    sol = solve(eqs, beta)
    G = sum(sol[beta[h]] * f[h] for h in range(N))
    F = G.subs(x, t / Vmax) / Vmax
    F_func_raw = lambdify(t, F, modules='numpy')

    def normalized_pdf(t_vals):
        vals = np.maximum(F_func_raw(t_vals), 0)
        sgrid = np.linspace(0, Vmax, 2000)
        norm_vals = np.maximum(F_func_raw(sgrid), 0)
        integral = np.trapz(norm_vals, sgrid)
        return vals / integral

    return normalized_pdf, F

# Interactive function
def update_plot(num_bins=22, poly_degree=10, spline_points=30):
    hist, bin_edges = np.histogram(wind_speed_data, bins=num_bins, density=True)
    bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
    ss_total = np.sum((hist - np.mean(hist))**2)

    # KDE
    start_kde = time.time()
    kde = gaussian_kde(wind_speed_data)
    kde_vals = kde(bin_centers)
    end_kde = time.time()
    time_kde = end_kde - start_kde

    # Chebyshev LS Fit
    t0 = time.time()
    cheb_ls_fit = Cheb.fit(bin_centers, hist, poly_degree-1, domain=[bin_edges[0], bin_edges[-1]])
    xls = np.linspace(bin_edges[0], bin_edges[-1], 2000)
    integral_ls = np.trapz(cheb_ls_fit(xls), xls)
    cheb_ls_norm = lambda x: cheb_ls_fit(x) / integral_ls
    cheb_ls_preds = cheb_ls_norm(bin_centers)
    time_chebls = time.time() - t0
    rmsq_chebls = np.sqrt(np.mean((cheb_ls_preds - hist)**2))
    r2_chebls = 1 - np.sum((hist - cheb_ls_preds)**2) / ss_total
    v = symbols('v')
    x = symbols('x')
    cheb_ls_expr = sum(c * v**i for i, c in enumerate(cheb_ls_fit.convert().coef))

    # Chebyshev Moment Fit
    start_mom = time.time()
    cheb_mom_func, cheb_mom_expr = FChebyshev(wind_speed_data, poly_degree)
    cheb_mom_preds = cheb_mom_func(bin_centers)
    end_mom = time.time()
    time_chebmom = end_mom - start_mom
    rmsq_chebmom = np.sqrt(np.mean((cheb_mom_preds - hist)**2))
    r2_chebmom = 1 - np.sum((hist - cheb_mom_preds)**2) / ss_total

    # Uniform Cubic Spline Fit clamped to zero slopes at both ends
    interp_x = np.linspace(bin_edges[0], bin_edges[-1], spline_points)
    spline_y = kde(interp_x)
    start_spline = time.time()
    spline = CubicSpline(interp_x, spline_y, bc_type=((1, 0.0), (1, 0.0)))
    spline_preds = spline(bin_centers)
    end_spline = time.time()
    time_spline = end_spline - start_spline
    rmsq_spline = np.sqrt(np.mean((hist - spline_preds)**2))
    r2_spline = 1 - np.sum((hist - spline_preds)**2) / ss_total

    # Symbolic Spline expression
    spline_expr = []
    for i in range(len(interp_x) - 1):
        a, b, c, d = spline.c[:, i]
        xi = interp_x[i]
        expr = a + b*(x - xi) + c*(x - xi)**2 + d*(x - xi)**3
        spline_expr.append((simplify(expr), (x >= xi) & (x < interp_x[i + 1])))
    spline_piecewise = Piecewise(*spline_expr)

    # Output metrics and expressions
    print(f"\n--- Fit Results ---")
    print(f"Chebyshev LS Fit:     RMSQ={rmsq_chebls:.4f}, R²={r2_chebls:.4f}, Time={time_chebls:.4f}s")
    print(f"  Expression: {latex(cheb_ls_expr)}\n")
    print(f"Chebyshev Moment Fit: RMSQ={rmsq_chebmom:.4f}, R²={r2_chebmom:.4f}, Time={time_chebmom:.4f}s")
    print(f"  Expression: {latex(cheb_mom_expr)}\n")
    print(f"KDE Estimate:         Time={time_kde:.4f}s")
    print(f"Cubic Spline Fit:     RMSQ={rmsq_spline:.4f}, R²={r2_spline:.4f}, Time={time_spline:.4f}s")
    print(f"  Symbolic Expression (Piecewise):\n{latex(spline_piecewise)}")

    # Plot
    x_range = np.linspace(bin_edges[0], bin_edges[-1], 1000)
    plt.figure(figsize=(12, 8), dpi=300)
    plt.hist(wind_speed_data, bins=num_bins, density=True, alpha=0.5, color='gray', edgecolor='black')
    plt.plot(x_range, cheb_ls_norm(x_range), 'g-.', label='Chebyshev LS Fit')
    plt.plot(x_range, cheb_mom_func(x_range), 'red', label='Chebyshev Moment')
    plt.plot(x_range, spline(x_range), 'purple', label='Cubic Spline')
    plt.plot(x_range, kde(x_range), 'orange', label='KDE')
    plt.xlabel('Wind Speed (m/s)')
    plt.ylabel('Density')
    plt.title(f"MERRA_basic_W34.666_S07.000_1992: R$^2$ = {r2_chebmom:.4f}, Degree = {poly_degree}, Bins = {num_bins}")
    plt.legend()
    plt.grid(True)
    plt.savefig("pdf_fit_comparison_highres.png", dpi=300)
    plt.show()

# Create interactive sliders
interact(update_plot,
         num_bins=IntSlider(min=5, max=50, step=1, value=23, description='Bins'),
         poly_degree=IntSlider(min=2, max=20, step=1, value=11, description='Poly Deg'),
         spline_points=IntSlider(min=5, max=100, step=1, value=15, description='Spline Nodes'))


interactive(children=(IntSlider(value=23, description='Bins', max=50, min=5), IntSlider(value=11, description=…

<function __main__.update_plot(num_bins=22, poly_degree=10, spline_points=30)>